# Class imbalanced 

- [ ] class weight
- [ ] synthetic data
- [ ] resampling

## class weight 

In [2]:
def get_classweight(trdl):
    
    labels = []
    for i in range(len(trdl.dataset)):
        _,y = trdl.dataset[i]
        labels.append(y)
    labels = np.array(labels)
    
    from sklearn.utils import class_weight
    class_weights=class_weight.compute_class_weight('balanced',classes=np.unique(tr_y),y=tr_y.to_numpy())
    class_weights=torch.tensor(class_weights,dtype=torch.float)
    return class_weights

In [2]:
from utils import get_classweight
import torch
from torch.utils.data import DataLoader
from dataset import VitalDataset_fs
idx = 3
root_dir = f'../data/all_{idx}'
trdt  = VitalDataset_fs(root_dir,f'../data/pd_gy/train_{idx}.json')
valdt = VitalDataset_fs(root_dir,f'../data/pd_gy/val_{idx}.json') 
tedt  = VitalDataset_fs(root_dir,f'../data/pd_gy/test_{idx}.json') 

trdl  = torch.utils.data.DataLoader(trdt,4)
valdl  = torch.utils.data.DataLoader(valdt,4)
tedl  = torch.utils.data.DataLoader(valdt,4)

from models import PrePostNet
from train import get_classweight
model = PrePostNet()

from torch import nn
device     = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 4

class_weight = get_classweight(trdl).to(device)
loss       = nn.CrossEntropyLoss(class_weight)
# loss       = nn.CrossEntropyLoss()
lr         = 0.01 


params = [p for p in model.parameters() if p.requires_grad]
opt  = torch.optim.Adam(params,lr=lr)

In [2]:
from train import trainval
best_model,val_losses = trainval(trdl,valdl,model,loss,opt,device=device, exist_acc=True)

[0] acc/loss: 0.83/0.183891
[1] acc/loss: 0.72/0.157175
[2] acc/loss: 0.69/0.288041
[3] acc/loss: 0.76/0.183061
[4] acc/loss: 0.83/0.154535
[5] acc/loss: 0.83/0.170922
[6] acc/loss: 0.83/0.158942
[7] acc/loss: 0.66/0.262116
[8] acc/loss: 0.7/0.210072
[9] acc/loss: 0.75/0.192625


## Weighted Random Sampler 

In [14]:
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler
from dataset import VitalDataset_fs
from models import PrePostNet
from train import get_classweight
from torch import nn

model = PrePostNet()
device     = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 4
# loss       = nn.CrossEntropyLoss()
lr         = 0.01 
params = [p for p in model.parameters() if p.requires_grad]
opt  = torch.optim.Adam(params,lr=lr)

idx = 3
root_dir = f'../data/all_{idx}'
trdt  = VitalDataset_fs(root_dir,f'../data/pd_gy/train_{idx}.json')
valdt = VitalDataset_fs(root_dir,f'../data/pd_gy/val_{idx}.json') 
tedt  = VitalDataset_fs(root_dir,f'../data/pd_gy/test_{idx}.json') 


class_weight = get_classweight(trdt).to(device)
loss       = nn.CrossEntropyLoss(class_weight)

from train import get_classweight_all
class_weight = torch.tensor([x/class_weight.sum() for x in class_weight])
class_weight_all = get_classweight_all(trdt,class_weight)

weighted_sampler = WeightedRandomSampler(
    weights=class_weight_all,
    num_samples=len(class_weight_all),
    replacement=True
)



trdl  = torch.utils.data.DataLoader(trdt,4,sampler=weighted_sampler)
valdl  = torch.utils.data.DataLoader(valdt,4)
tedl  = torch.utils.data.DataLoader(valdt,4)

In [13]:
torch.tensor([x/class_weight.sum() for x in class_weight])

tensor([0.1729, 0.8271])

In [15]:
from train import trainval
best_model,val_losses = trainval(trdl,valdl,model,loss,opt,device=device, exist_acc=True)

[0] acc/loss: 0.17/0.494183
[1] acc/loss: 0.17/0.314152
[2] acc/loss: 0.17/0.243351
[3] acc/loss: 0.17/0.220442
[4] acc/loss: 0.17/0.457828
[5] acc/loss: 0.37/0.213405
[6] acc/loss: 0.17/0.535542
[7] acc/loss: 0.17/1.223115
[8] acc/loss: 0.19/0.236616
[9] acc/loss: 0.18/0.236128
[10] acc/loss: 0.32/0.234474


In [4]:
class_weight

tensor([0.6045, 2.8917], device='cuda:0')

In [45]:
def get_classweight_all(trdt,class_weight):
    from torch.utils.data import WeightedRandomSampler
    tr_y = []
    for i in range(len(trdt)):
        tr_y.append(trdt[i][1])
    class_weight_all = class_weight[tr_y]
    return class_weight_all

In [ ]:
from train import get_classweight_all
get_classweight_all(trdt,class_weight)
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

tensor([2.8917, 2.8917, 0.6045, 0.6045, 0.6045, 2.8917, 0.6045, 0.6045, 0.6045,
        0.6045, 2.8917, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045,
        0.6045, 2.8917, 0.6045, 2.8917, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045,
        0.6045, 0.6045, 0.6045, 0.6045, 2.8917, 0.6045, 0.6045, 2.8917, 2.8917,
        0.6045, 0.6045, 2.8917, 2.8917, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045,
        0.6045, 0.6045, 0.6045, 2.8917, 2.8917, 2.8917, 0.6045, 0.6045, 0.6045,
        2.8917, 0.6045, 2.8917, 0.6045, 0.6045, 0.6045, 2.8917, 0.6045, 0.6045,
        2.8917, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045,
        0.6045, 0.6045, 0.6045, 2.8917, 0.6045, 0.6045, 0.6045, 2.8917, 0.6045,
        0.6045, 0.6045, 0.6045, 2.8917, 0.6045, 0.6045, 0.6045, 0.6045, 2.8917,
        2.8917, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 2.8917,
        0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045, 0.6045,
        0.6045, 0.6045, 0.6045, 0.6045, 

In [43]:
from torch.utils.data import WeightedRandomSampler
tr_y = []
for i in range(len(trdt)):
    tr_y.append(trdt[i][1])

class_weights_all = class_weight[tr_y]

weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)